<a href="https://colab.research.google.com/github/SahilLokhande2604/Crop_Yield_Prediction/blob/main/Crop_Yield_Using_Shap_Feature_Extraction_for_top_10_crops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Inportance using shap for top 10 crops


In [ ]:
!pip install scikeras

In [ ]:
!pip install shap

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras import layers, models
import shap
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('crop_yield.csv')

# Identifying features and target
target = 'Yield'
features = ['Crop', 'Crop_Year', 'Season', 'State', 'Area', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']

X = data[features]
y = data[target]

# Splitting data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing Pipeline:
# 1. OneHotEncode categorical variables
# 2. Standardize numerical variables
categorical_features = ['Crop', 'Season', 'State']
numerical_features = ['Crop_Year', 'Area', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']

# Define Column Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Fit and transform the training data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Define the neural network architecture
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_preprocessed.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train_preprocessed, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 584726.1250 - mae: 72.7376 - val_loss: 515559.7812 - val_mae: 126.5494
Epoch 2/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 263190.1875 - mae: 75.8955 - val_loss: 61837.9531 - val_mae: 37.7552
Epoch 3/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 85602.2344 - mae: 33.0691 - val_loss: 36338.7578 - val_mae: 29.2731
Epoch 4/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 30205.0957 - mae: 23.1566 - val_loss: 26861.1465 - val_mae: 21.7608
Epoch 5/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 45976.1250 - mae: 22.3057 - val_loss: 24246.0020 - val_mae: 19.9377
Epoch 6/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 28180.9629 - mae: 19.1818 - val_loss: 21357.7988 - val_mae: 19.2687
Epoch 7/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 23181.8379 - mae: 17.2244 - val_loss: 20305.0273 - val_mae: 17.8836
Epoch 8/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 20028.2656 - mae: 15.6844 - val_loss: 19

In [ ]:
import numpy as np
# Get user input for the crop name
crop_name = input("Enter the name of the crop you want to analyze: ")

print(f"Analyzing feature importance for {crop_name}")

Enter the name of the crop you want to analyze: Rice
Analyzing feature importance for Rice


In [ ]:

# Subset data for the specific crop
crop_data = data[data['Crop'] == crop_name]

# Skip crops with too few samples
if len(crop_data) < 10:
    print(f"Skipping {crop_name} due to insufficient data (less than 10 samples)")
else:
    # Preprocess the crop-specific data
    X_crop = crop_data[features]
    X_crop_preprocessed = preprocessor.transform(X_crop)

    # Use k-means to summarize the background data (e.g., to 10 clusters)
    # Use shape[0] to get the number of rows in the sparse matrix
    num_clusters = min(10, X_crop_preprocessed.shape[0])  # Use fewer clusters if the crop data is small
    background_crop = shap.kmeans(X_crop_preprocessed, num_clusters)  # Summarize the background data

    # SHAP explainer with summarized background
    explainer = shap.KernelExplainer(model.predict, background_crop)


# Compute SHAP values for the crop-specific data
shap_values_crop = explainer.shap_values(X_crop_preprocessed)

# Get SHAP values as a NumPy array for easier manipulation
shap_values_array = np.array(shap_values_crop)

# Get original feature names from the ColumnTransformer
original_feature_names = numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))

# Calculate mean absolute SHAP values for each feature, and flatten the array
mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0).flatten()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


  0%|          | 0/1197 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
320/320 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
640/640 ━━━━━

In [ ]:
import plotly.graph_objects as go

# Assuming original_feature_names and mean_abs_shap_values are defined

# Create a bar plot using Plotly
fig = go.Figure()

# Add bars to the figure
fig.add_trace(go.Bar(
    x=original_feature_names,
    y=mean_abs_shap_values,
    marker=dict(color='blue'),
    hovertemplate='<b>%{x}</b><br>Mean Absolute SHAP Value: %{y}<extra></extra>',
))

# Update layout
fig.update_layout(
    title=f"Feature Importance for {crop_name}",
    xaxis_title="Features",
    yaxis_title="Mean Absolute SHAP Value",
    xaxis_tickangle=-45,  # Rotate x-axis labels
)

# Show the plot
fig.show()


# Turmeric

In [ ]:
import numpy as np
# Get user input for the crop name
crop_name = input("Enter the name of the crop you want to analyze: ")

print(f"Analyzing feature importance for {crop_name}")

Enter the name of the crop you want to analyze: Turmeric
Analyzing feature importance for Turmeric


In [ ]:

# Subset data for the specific crop
crop_data = data[data['Crop'] == crop_name]

# Skip crops with too few samples
if len(crop_data) < 10:
    print(f"Skipping {crop_name} due to insufficient data (less than 10 samples)")
else:
    # Preprocess the crop-specific data
    X_crop = crop_data[features]
    X_crop_preprocessed = preprocessor.transform(X_crop)

    # Use k-means to summarize the background data (e.g., to 10 clusters)
    # Use shape[0] to get the number of rows in the sparse matrix
    num_clusters = min(10, X_crop_preprocessed.shape[0])  # Use fewer clusters if the crop data is small
    background_crop = shap.kmeans(X_crop_preprocessed, num_clusters)  # Summarize the background data

    # SHAP explainer with summarized background
    explainer = shap.KernelExplainer(model.predict, background_crop)


# Compute SHAP values for the crop-specific data
shap_values_crop = explainer.shap_values(X_crop_preprocessed)

# Get SHAP values as a NumPy array for easier manipulation
shap_values_array = np.array(shap_values_crop)

# Get original feature names from the ColumnTransformer
original_feature_names = numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))

# Calculate mean absolute SHAP values for each feature, and flatten the array
mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0).flatten()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step


  0%|          | 0/337 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
649/649 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
649/649 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
649/649 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
649/649 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
649/649 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
649/649 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
649/649 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
649/649 ━━━

In [ ]:
import plotly.graph_objects as go

# Assuming original_feature_names and mean_abs_shap_values are defined

# Create a bar plot using Plotly
fig = go.Figure()

# Add bars to the figure
fig.add_trace(go.Bar(
    x=original_feature_names,
    y=mean_abs_shap_values,
    marker=dict(color='blue'),
    hovertemplate='<b>%{x}</b><br>Mean Absolute SHAP Value: %{y}<extra></extra>',
))

# Update layout
fig.update_layout(
    title=f"Feature Importance for {crop_name}",
    xaxis_title="Features",
    yaxis_title="Mean Absolute SHAP Value",
    xaxis_tickangle=-45,  # Rotate x-axis labels
)

# Show the plot
fig.show()


# Sugarcane

In [ ]:
import numpy as np
# Get user input for the crop name
crop_name = input("Enter the name of the crop you want to analyze: ")

print(f"Analyzing feature importance for {crop_name}")

Enter the name of the crop you want to analyze: Sugarcane
Analyzing feature importance for Sugarcane


In [ ]:

# Subset data for the specific crop
crop_data = data[data['Crop'] == crop_name]

# Skip crops with too few samples
if len(crop_data) < 10:
    print(f"Skipping {crop_name} due to insufficient data (less than 10 samples)")
else:
    # Preprocess the crop-specific data
    X_crop = crop_data[features]
    X_crop_preprocessed = preprocessor.transform(X_crop)

    # Use k-means to summarize the background data (e.g., to 10 clusters)
    # Use shape[0] to get the number of rows in the sparse matrix
    num_clusters = min(10, X_crop_preprocessed.shape[0])  # Use fewer clusters if the crop data is small
    background_crop = shap.kmeans(X_crop_preprocessed, num_clusters)  # Summarize the background data

    # SHAP explainer with summarized background
    explainer = shap.KernelExplainer(model.predict, background_crop)


# Compute SHAP values for the crop-specific data
shap_values_crop = explainer.shap_values(X_crop_preprocessed)

# Get SHAP values as a NumPy array for easier manipulation
shap_values_array = np.array(shap_values_crop)

# Get original feature names from the ColumnTransformer
original_feature_names = numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))

# Calculate mean absolute SHAP values for each feature, and flatten the array
mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0).flatten()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


  0%|          | 0/605 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
640/640 ━━━━━

In [ ]:
import plotly.graph_objects as go

# Assuming original_feature_names and mean_abs_shap_values are defined

# Create a bar plot using Plotly
fig = go.Figure()

# Add bars to the figure
fig.add_trace(go.Bar(
    x=original_feature_names,
    y=mean_abs_shap_values,
    marker=dict(color='blue'),
    hovertemplate='<b>%{x}</b><br>Mean Absolute SHAP Value: %{y}<extra></extra>',
))

# Update layout
fig.update_layout(
    title=f"Feature Importance for {crop_name}",
    xaxis_title="Features",
    yaxis_title="Mean Absolute SHAP Value",
    xaxis_tickangle=-45,  # Rotate x-axis labels
)

# Show the plot
fig.show()


# Castor seed

In [ ]:
import numpy as np
# Get user input for the crop name
crop_name = input("Enter the name of the crop you want to analyze: ")

print(f"Analyzing feature importance for {crop_name}")

Enter the name of the crop you want to analyze: Castor seed
Analyzing feature importance for Castor seed


In [ ]:

# Subset data for the specific crop
crop_data = data[data['Crop'] == crop_name]

# Skip crops with too few samples
if len(crop_data) < 10:
    print(f"Skipping {crop_name} due to insufficient data (less than 10 samples)")
else:
    # Preprocess the crop-specific data
    X_crop = crop_data[features]
    X_crop_preprocessed = preprocessor.transform(X_crop)

    # Use k-means to summarize the background data (e.g., to 10 clusters)
    # Use shape[0] to get the number of rows in the sparse matrix
    num_clusters = min(10, X_crop_preprocessed.shape[0])  # Use fewer clusters if the crop data is small
    background_crop = shap.kmeans(X_crop_preprocessed, num_clusters)  # Summarize the background data

    # SHAP explainer with summarized background
    explainer = shap.KernelExplainer(model.predict, background_crop)


# Compute SHAP values for the crop-specific data
shap_values_crop = explainer.shap_values(X_crop_preprocessed)

# Get SHAP values as a NumPy array for easier manipulation
shap_values_array = np.array(shap_values_crop)

# Get original feature names from the ColumnTransformer
original_feature_names = numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))

# Calculate mean absolute SHAP values for each feature, and flatten the array
mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0).flatten()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


  0%|          | 0/300 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
649/649 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
640/640 ━━━━━━

In [ ]:
import plotly.graph_objects as go

# Assuming original_feature_names and mean_abs_shap_values are defined

# Create a bar plot using Plotly
fig = go.Figure()

# Add bars to the figure
fig.add_trace(go.Bar(
    x=original_feature_names,
    y=mean_abs_shap_values,
    marker=dict(color='blue'),
    hovertemplate='<b>%{x}</b><br>Mean Absolute SHAP Value: %{y}<extra></extra>',
))

# Update layout
fig.update_layout(
    title=f"Feature Importance for {crop_name}",
    xaxis_title="Features",
    yaxis_title="Mean Absolute SHAP Value",
    xaxis_tickangle=-45,  # Rotate x-axis labels
)

# Show the plot
fig.show()


# Cashewnut

In [ ]:
import numpy as np
# Get user input for the crop name
crop_name = input("Enter the name of the crop you want to analyze: ")

print(f"Analyzing feature importance for {crop_name}")

Enter the name of the crop you want to analyze: Cashewnut
Analyzing feature importance for Cashewnut


In [ ]:

# Subset data for the specific crop
crop_data = data[data['Crop'] == crop_name]

# Skip crops with too few samples
if len(crop_data) < 10:
    print(f"Skipping {crop_name} due to insufficient data (less than 10 samples)")
else:
    # Preprocess the crop-specific data
    X_crop = crop_data[features]
    X_crop_preprocessed = preprocessor.transform(X_crop)

    # Use k-means to summarize the background data (e.g., to 10 clusters)
    # Use shape[0] to get the number of rows in the sparse matrix
    num_clusters = min(10, X_crop_preprocessed.shape[0])  # Use fewer clusters if the crop data is small
    background_crop = shap.kmeans(X_crop_preprocessed, num_clusters)  # Summarize the background data

    # SHAP explainer with summarized background
    explainer = shap.KernelExplainer(model.predict, background_crop)


# Compute SHAP values for the crop-specific data
shap_values_crop = explainer.shap_values(X_crop_preprocessed)

# Get SHAP values as a NumPy array for easier manipulation
shap_values_array = np.array(shap_values_crop)

# Get original feature names from the ColumnTransformer
original_feature_names = numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))

# Calculate mean absolute SHAP values for each feature, and flatten the array
mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0).flatten()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


  0%|          | 0/134 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
650/650 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
650/650 ━━━

In [ ]:
import plotly.graph_objects as go

# Assuming original_feature_names and mean_abs_shap_values are defined

# Create a bar plot using Plotly
fig = go.Figure()

# Add bars to the figure
fig.add_trace(go.Bar(
    x=original_feature_names,
    y=mean_abs_shap_values,
    marker=dict(color='blue'),
    hovertemplate='<b>%{x}</b><br>Mean Absolute SHAP Value: %{y}<extra></extra>',
))

# Update layout
fig.update_layout(
    title=f"Feature Importance for {crop_name}",
    xaxis_title="Features",
    yaxis_title="Mean Absolute SHAP Value",
    xaxis_tickangle=-45,  # Rotate x-axis labels
)

# Show the plot
fig.show()


# Tobacco

In [ ]:
import numpy as np
# Get user input for the crop name
crop_name = input("Enter the name of the crop you want to analyze: ")

print(f"Analyzing feature importance for {crop_name}")

Enter the name of the crop you want to analyze: Tobacco
Analyzing feature importance for Tobacco


In [ ]:

# Subset data for the specific crop
crop_data = data[data['Crop'] == crop_name]

# Skip crops with too few samples
if len(crop_data) < 10:
    print(f"Skipping {crop_name} due to insufficient data (less than 10 samples)")
else:
    # Preprocess the crop-specific data
    X_crop = crop_data[features]
    X_crop_preprocessed = preprocessor.transform(X_crop)

    # Use k-means to summarize the background data (e.g., to 10 clusters)
    # Use shape[0] to get the number of rows in the sparse matrix
    num_clusters = min(10, X_crop_preprocessed.shape[0])  # Use fewer clusters if the crop data is small
    background_crop = shap.kmeans(X_crop_preprocessed, num_clusters)  # Summarize the background data

    # SHAP explainer with summarized background
    explainer = shap.KernelExplainer(model.predict, background_crop)


# Compute SHAP values for the crop-specific data
shap_values_crop = explainer.shap_values(X_crop_preprocessed)

# Get SHAP values as a NumPy array for easier manipulation
shap_values_array = np.array(shap_values_crop)

# Get original feature names from the ColumnTransformer
original_feature_names = numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))

# Calculate mean absolute SHAP values for each feature, and flatten the array
mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0).flatten()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


  0%|          | 0/364 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
648/648 ━━━━━

In [ ]:
import plotly.graph_objects as go

# Assuming original_feature_names and mean_abs_shap_values are defined

# Create a bar plot using Plotly
fig = go.Figure()

# Add bars to the figure
fig.add_trace(go.Bar(
    x=original_feature_names,
    y=mean_abs_shap_values,
    marker=dict(color='blue'),
    hovertemplate='<b>%{x}</b><br>Mean Absolute SHAP Value: %{y}<extra></extra>',
))

# Update layout
fig.update_layout(
    title=f"Feature Importance for {crop_name}",
    xaxis_title="Features",
    yaxis_title="Mean Absolute SHAP Value",
    xaxis_tickangle=-45,  # Rotate x-axis labels
)

# Show the plot
fig.show()


# Maize

In [ ]:
import numpy as np
# Get user input for the crop name
crop_name = input("Enter the name of the crop you want to analyze: ")

print(f"Analyzing feature importance for {crop_name}")

Enter the name of the crop you want to analyze: Maize
Analyzing feature importance for Maize


In [ ]:

# Subset data for the specific crop
crop_data = data[data['Crop'] == crop_name]

# Skip crops with too few samples
if len(crop_data) < 10:
    print(f"Skipping {crop_name} due to insufficient data (less than 10 samples)")
else:
    # Preprocess the crop-specific data
    X_crop = crop_data[features]
    X_crop_preprocessed = preprocessor.transform(X_crop)

    # Use k-means to summarize the background data (e.g., to 10 clusters)
    # Use shape[0] to get the number of rows in the sparse matrix
    num_clusters = min(10, X_crop_preprocessed.shape[0])  # Use fewer clusters if the crop data is small
    background_crop = shap.kmeans(X_crop_preprocessed, num_clusters)  # Summarize the background data

    # SHAP explainer with summarized background
    explainer = shap.KernelExplainer(model.predict, background_crop)


# Compute SHAP values for the crop-specific data
shap_values_crop = explainer.shap_values(X_crop_preprocessed)

# Get SHAP values as a NumPy array for easier manipulation
shap_values_array = np.array(shap_values_crop)

# Get original feature names from the ColumnTransformer
original_feature_names = numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))

# Calculate mean absolute SHAP values for each feature, and flatten the array
mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0).flatten()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


  0%|          | 0/975 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
640/640 ━━━━━━

In [ ]:
import plotly.graph_objects as go

# Assuming original_feature_names and mean_abs_shap_values are defined

# Create a bar plot using Plotly
fig = go.Figure()

# Add bars to the figure
fig.add_trace(go.Bar(
    x=original_feature_names,
    y=mean_abs_shap_values,
    marker=dict(color='blue'),
    hovertemplate='<b>%{x}</b><br>Mean Absolute SHAP Value: %{y}<extra></extra>',
))

# Update layout
fig.update_layout(
    title=f"Feature Importance for {crop_name}",
    xaxis_title="Features",
    yaxis_title="Mean Absolute SHAP Value",
    xaxis_tickangle=-45,  # Rotate x-axis labels
)

# Show the plot
fig.show()


In [ ]:
import numpy as np
# Get user input for the crop name
crop_name = input("Enter the name of the crop you want to analyze: ")

print(f"Analyzing feature importance for {crop_name}")

In [ ]:

# Subset data for the specific crop
crop_data = data[data['Crop'] == crop_name]

# Skip crops with too few samples
if len(crop_data) < 10:
    print(f"Skipping {crop_name} due to insufficient data (less than 10 samples)")
else:
    # Preprocess the crop-specific data
    X_crop = crop_data[features]
    X_crop_preprocessed = preprocessor.transform(X_crop)

    # Use k-means to summarize the background data (e.g., to 10 clusters)
    # Use shape[0] to get the number of rows in the sparse matrix
    num_clusters = min(10, X_crop_preprocessed.shape[0])  # Use fewer clusters if the crop data is small
    background_crop = shap.kmeans(X_crop_preprocessed, num_clusters)  # Summarize the background data

    # SHAP explainer with summarized background
    explainer = shap.KernelExplainer(model.predict, background_crop)


# Compute SHAP values for the crop-specific data
shap_values_crop = explainer.shap_values(X_crop_preprocessed)

# Get SHAP values as a NumPy array for easier manipulation
shap_values_array = np.array(shap_values_crop)

# Get original feature names from the ColumnTransformer
original_feature_names = numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))

# Calculate mean absolute SHAP values for each feature, and flatten the array
mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0).flatten()


In [ ]:
import plotly.graph_objects as go

# Assuming original_feature_names and mean_abs_shap_values are defined

# Create a bar plot using Plotly
fig = go.Figure()

# Add bars to the figure
fig.add_trace(go.Bar(
    x=original_feature_names,
    y=mean_abs_shap_values,
    marker=dict(color='blue'),
    hovertemplate='<b>%{x}</b><br>Mean Absolute SHAP Value: %{y}<extra></extra>',
))

# Update layout
fig.update_layout(
    title=f"Feature Importance for {crop_name}",
    xaxis_title="Features",
    yaxis_title="Mean Absolute SHAP Value",
    xaxis_tickangle=-45,  # Rotate x-axis labels
)

# Show the plot
fig.show()


In [ ]:
import numpy as np
# Get user input for the crop name
crop_name = input("Enter the name of the crop you want to analyze: ")

print(f"Analyzing feature importance for {crop_name}")

In [ ]:

# Subset data for the specific crop
crop_data = data[data['Crop'] == crop_name]

# Skip crops with too few samples
if len(crop_data) < 10:
    print(f"Skipping {crop_name} due to insufficient data (less than 10 samples)")
else:
    # Preprocess the crop-specific data
    X_crop = crop_data[features]
    X_crop_preprocessed = preprocessor.transform(X_crop)

    # Use k-means to summarize the background data (e.g., to 10 clusters)
    # Use shape[0] to get the number of rows in the sparse matrix
    num_clusters = min(10, X_crop_preprocessed.shape[0])  # Use fewer clusters if the crop data is small
    background_crop = shap.kmeans(X_crop_preprocessed, num_clusters)  # Summarize the background data

    # SHAP explainer with summarized background
    explainer = shap.KernelExplainer(model.predict, background_crop)


# Compute SHAP values for the crop-specific data
shap_values_crop = explainer.shap_values(X_crop_preprocessed)

# Get SHAP values as a NumPy array for easier manipulation
shap_values_array = np.array(shap_values_crop)

# Get original feature names from the ColumnTransformer
original_feature_names = numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))

# Calculate mean absolute SHAP values for each feature, and flatten the array
mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0).flatten()


In [ ]:
import plotly.graph_objects as go

# Assuming original_feature_names and mean_abs_shap_values are defined

# Create a bar plot using Plotly
fig = go.Figure()

# Add bars to the figure
fig.add_trace(go.Bar(
    x=original_feature_names,
    y=mean_abs_shap_values,
    marker=dict(color='blue'),
    hovertemplate='<b>%{x}</b><br>Mean Absolute SHAP Value: %{y}<extra></extra>',
))

# Update layout
fig.update_layout(
    title=f"Feature Importance for {crop_name}",
    xaxis_title="Features",
    yaxis_title="Mean Absolute SHAP Value",
    xaxis_tickangle=-45,  # Rotate x-axis labels
)

# Show the plot
fig.show()
